In [2]:
from preamble import *
import pandas as pd   # pandas 불러오기

In [4]:
df = pd.read_csv("data/housing.csv")
df = df.dropna()
df.head()
df.info()

FileNotFoundError: [Errno 2] No such file or directory: 'data/housing.csv'

## 데이터 전처리

In [ ]:
## X와 y
X = df.drop("median_house_value", axis=1)
y = df["median_house_value"]

## 수치형과 범주형으로 데이터를 나눠보세요.
# num_features = X.drop("ocean_proximity")
# cat_features = X["ocean_proximity"]

## 수치형과 범주형으로 데이터를 나눠보세요(리스트로 만드세요!)
num_features = X.select_dtypes(include=["float64", "int64", "float32", "int32"]).columns.to_list()
cat_features = ["ocean_proximity"]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train_num = X_train[num_features]
X_train_cat = X_train[cat_features]
X_test_num = X_test[num_features]
X_test_cat = X_test[cat_features]

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_num_scaled = scaler.fit_transform(X_train_num)
x_test_num_scaled = scaler.transform(X_test_num)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
X_train_cat_encoder = encoder.fit_transform(X_train_cat).toarray()
X_test_cat_encoder = encoder.transform(X_test_cat).toarray()

In [ ]:
X_train_processed = np.hstack([x_train_num_scaled, X_train_cat_encoder])
X_test_processed = np.hstack([x_test_num_scaled, X_test_cat_encoder])

## 모델링

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

def evaluate_model(name, y_true, y_pred):
    r2 = r2_score(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    print(f"{name}")
    print(f" -- r2   : {r2:.2f}")
    print(f" -- RMSE : {rmse:.2f}")
    return {"model": name, "r2": r2, "rsme": rmse}

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train_processed, y_train)
lr_ypred = lr.predict(X_test_processed)

result_lr = evaluate_model("Linear Regression", y_test, lr_ypred)
result_lr

Linear Regression
 -- r2   : 0.65
 -- RMSE : 69297.72


{'model': 'Linear Regression',
 'r2': 0.6488402154431994,
 'rsme': np.float64(69297.7166911303)}

In [ ]:
# 범주형은 제외해야 합니다. 수치형만 가능해요!
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_num_poly = poly.fit_transform(x_train_num_scaled)
X_test_num_poly = poly.transform(x_test_num_scaled)

In [ ]:
X_train_poly_processed = np.hstack([X_train_num_poly, X_train_cat_encoder])
X_test_poly_processed = np.hstack([X_test_num_poly, X_test_cat_encoder])

In [ ]:
lr.fit(X_train_poly_processed, y_train)
lr_ypred = lr.predict(X_test_poly_processed)
result_lr = evaluate_model("Linear Regression (Poly)", y_test, lr_ypred)
result_lr

Linear Regression (Poly)
 -- r2   : 0.70
 -- RMSE : 64014.26


{'model': 'Linear Regression (Poly)',
 'r2': 0.7003458266153706,
 'rsme': np.float64(64014.25618039147)}